Consider a 5-month American put option on a non-dividend-paying stock when
the stock price is 50, the strike price is 50, the risk-free interest rate is 10% per
annum, and the volatility is 40% per annum. With our usual notation, this means
that S0 = 50, K = 50, r = 0.10, s = 0.40, T = 0.4167, and q = 0.

Values of 20, 10, and 5 were chosen for M, N, and ΔS, respectively. Thus, the option price is
evaluated at dollar5 stock price intervals between dollar0 and dollar100 and at half-month time
intervals throughout the life of the option. The option price given by the grid is
dollar4.07.
The same grid gives the price of the corresponding European option as
dollar3.91

In [1]:
import numpy as np
import scipy.linalg as linalg
from numpy import exp
import pandas as pd

In [2]:
def ImplicitFDS(S0,K,r,T,sigma,Smax,M,N,AMvsEU,CALLvsPUT):
    dS=Smax/float(M)
    dt=T/float(N)
    i_values=np.arange(M)
    j_values=np.arange(N)
    grid=np.zeros(shape=(M+1,N+1))
    boundary_conds=np.linspace(0,Smax,M+1)
    
    if CALLvsPUT=='CALL':
        grid[:,-1]= np.maximum(boundary_conds-K,0)
        grid[-1,:-1]= Smax - K*exp(-r*dt*(N-j_values))  
        if AMvsEU=='AM':
            grid[-1,:-1]= np.maximum(grid[-1,:-1],Smax-K)
    if CALLvsPUT=='PUT':
        grid[:,-1]= np.maximum(K-boundary_conds,0)
        grid[0,:-1]=K*exp(-r*dt*(N-j_values))
        if AMvsEU=='AM':
            grid[0,:-1]= np.maximum(grid[-1,:-1],K)
        
    a= 0.5*dt*((r*i_values) - (sigma**2 * i_values**2))
    b= 1 + dt*((sigma**2 * i_values**2) + r)
    c= -0.5*dt*((r*i_values) + (sigma**2 * i_values**2))
    coeffs= np.diag(a[2:M],-1) + np.diag(b[1:M]) + np.diag(c[1:M-1],1)
    
    P, L, U = linalg.lu(coeffs)
    aux = np.zeros(M-1)
    for j in reversed(range(N)):
        aux[0] = np.dot(a[1], grid[0, j])
        aux[-1]= np.dot(c[-1],grid[-1,j])
        x1 = linalg.solve(L, grid[1:M, j+1]-aux)
        x2 = linalg.solve(U, x1)
        grid[1:M, j] = x2  
        if AMvsEU=='AM':
            grid[1:M, j]=np.maximum(grid[1:M, j],grid[1:M, -1])
            
    dataset={'Stock price':boundary_conds,'t=0':np.round(grid[:,0],2),'t=0.5/12':np.round(grid[:,1],2),
             't=1/12':np.round(grid[:,2],2),'t=1.5/12':np.round(grid[:,3],2),'t=2/12':np.round(grid[:,4],2),
             't=2.5/12':np.round(grid[:,5],2),'t=3/12':np.round(grid[:,6],2),'t=3.5/12':np.round(grid[:,7],2),
             't=4/12':np.round(grid[:,8],2),'t=4.5/12':np.round(grid[:,9],2),'t=5/12':np.round(grid[:,10],2)}
    return pd.DataFrame(dataset)


In [3]:
print(ImplicitFDS(50,50,0.10,5/12,0.40,100,20,10,'AM','PUT'))

    Stock price    t=0  t=0.5/12  t=1/12  t=1.5/12  t=2/12  t=2.5/12  t=3/12  \
0           0.0  50.00     50.00   50.00     50.00   50.00     50.00   50.00   
1           5.0  45.00     45.00   45.00     45.00   45.00     45.00   45.00   
2          10.0  40.00     40.00   40.00     40.00   40.00     40.00   40.00   
3          15.0  35.00     35.00   35.00     35.00   35.00     35.00   35.00   
4          20.0  30.00     30.00   30.00     30.00   30.00     30.00   30.00   
5          25.0  25.00     25.00   25.00     25.00   25.00     25.00   25.00   
6          30.0  20.00     20.00   20.00     20.00   20.00     20.00   20.00   
7          35.0  15.00     15.00   15.00     15.00   15.00     15.00   15.00   
8          40.0  10.15     10.10   10.05     10.01   10.00     10.00   10.00   
9          45.0   6.58      6.44    6.29      6.13    5.96      5.77    5.57   
10         50.0   4.07      3.88    3.67      3.45    3.19      2.91    2.57   
11         55.0   2.43      2.24    2.05

European Option Valuation

In [4]:
def ImplicitFDS(S0,K,r,T,sigma,Smax,M,N,AMvsEU,CALLvsPUT):
    dS=Smax/float(M)
    dt=T/float(N)
    i_values=np.arange(M)
    j_values=np.arange(N)
    grid=np.zeros(shape=(M+1,N+1))
    boundary_conds=np.linspace(0,Smax,M+1)
    
    if CALLvsPUT=='CALL':
        grid[:,-1]= np.maximum(boundary_conds-K,0)
        grid[-1,:-1]= Smax - K*exp(-r*dt*(N-j_values))  
        if AMvsEU=='AM':
            grid[-1,:-1]= np.maximum(grid[-1,:-1],Smax-K)
    if CALLvsPUT=='PUT':
        grid[:,-1]= np.maximum(K-boundary_conds,0)
        grid[0,:-1]=K*exp(-r*dt*(N-j_values))
        if AMvsEU=='AM':
            grid[0,:-1]= np.maximum(grid[-1,:-1],K)
        
    a= 0.5*dt*((r*i_values) - (sigma**2 * i_values**2))
    b= 1 + dt*((sigma**2 * i_values**2) + r)
    c= -0.5*dt*((r*i_values) + (sigma**2 * i_values**2))
    coeffs= np.diag(a[2:M],-1) + np.diag(b[1:M]) + np.diag(c[1:M-1],1)
    
    P, L, U = linalg.lu(coeffs)
    aux = np.zeros(M-1)
    for j in reversed(range(N)):
        aux[0] = np.dot(a[1], grid[0, j])
        aux[-1]= np.dot(c[-1],grid[-1,j])
        x1 = linalg.solve(L, grid[1:M, j+1]-aux)
        x2 = linalg.solve(U, x1)
        grid[1:M, j] = x2  
        if AMvsEU=='AM':
            grid[1:M, j]=np.maximum(grid[1:M, j],grid[1:M, -1])
            
    return np.interp(S0,boundary_conds,grid[:,0])

In [5]:
print(ImplicitFDS(50,50,0.10,5/12,0.40,100,20,10,'EU','PUT'))

3.911207736211867


In [6]:
S0=50; K=50; r=0.10; T=5/12; sigma=0.40; Smax=100; M=20; N=10; CALLvsPUT='PUT'

In [7]:
dS=Smax/float(M)
dt=T/float(N)
i_values=np.arange(M)
j_values=np.arange(N)
grid=np.zeros(shape=(M+1,N+1))
boundary_conds=np.linspace(0,Smax,M+1)

In [8]:
if CALLvsPUT=='CALL':
    grid[:,-1]= np.maximum(boundary_conds-K,0)
    grid[-1,:-1]= Smax - K*exp(-r*dt*(N-j_values))  
if CALLvsPUT=='PUT':
    grid[:,-1]= np.maximum(K-boundary_conds,0)
    grid[0,:-1]=K*exp(-r*dt*(N-j_values))
    

In [9]:
a= 0.5*dt*((r*i_values) - (sigma**2 * i_values**2))
b= 1 + dt*((sigma**2 * i_values**2) + r)
c= -0.5*dt*((r*i_values) + (sigma**2 * i_values**2))

coeffs= np.diag(a[2:M],-1) + np.diag(b[1:M]) + np.diag(c[1:M-1],1)

In [10]:
P, L, U = linalg.lu(coeffs)
aux = np.zeros(M-1)
for j in reversed(range(N)):
    aux[0] = np.dot(-a[1], grid[0, j])
    x1 = linalg.solve(L, grid[1:M, j+1]+aux)
    x2 = linalg.solve(U, x1)
    grid[1:M, j] = x2            

In [11]:
np.interp(S0,boundary_conds,grid[:,0])

3.911207736211867